In [3]:
from pathlib import Path
import fitz  # PyMuPDF
import os
from dotenv import load_dotenv  # Optional for secure API key handling

from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

In [4]:
# ✅ Load environment variables from .env if it exists
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    raise ValueError("❌ OPENAI_API_KEY not found. Please set it in .env or environment variables.")

    
# ✅ Set up embedding model
embedding = OpenAIEmbeddings(openai_api_key=openai_api_key)

/var/folders/cq/tvx_t3ws7xxcsv5lz1_j4j_40000gn/T/ipykernel_10289/4034385780.py:10: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings(openai_api_key=openai_api_key)


In [5]:
# ✅ Step 1: Extract text from all PDFs
def extract_text_from_pdf(pdf_path):
    print(f"📄 Reading: {pdf_path.name}")
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

In [ ]:
# ✅ Load all text from /data folder
pdf_dir = Path("../data/tutorials")
pdf_files = list(pdf_dir.glob("*.pdf"))

if not pdf_files:
    raise FileNotFoundError("❌ No PDF files found in /data/tutorials folder.")

all_text = ""
for file in pdf_files:
    all_text += extract_text_from_pdf(file)

print("✅ All PDF content loaded.")

FileNotFoundError: ❌ No PDF files found in /data folder.

In [ ]:
# ✅ Step 2: Chunk the text
splitter = CharacterTextSplitter(chunk_size=800, chunk_overlap=100)
chunks = splitter.split_text(all_text)

print(f"✅ Total chunks created: {len(chunks)}")

In [ ]:
# ✅ Step 3: Embed and store in ChromaDB
persist_dir = "../db"
db = Chroma.from_texts(chunks, embedding, persist_directory=persist_dir)
db.persist()

print(f"✅ Vector database saved to {persist_dir}")